In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("Processed_Data.csv")

In [3]:
df

,Unnamed: 0,comment,label,num_character,num_sent,num_word,comment_transformed
0,0,dalits are lowlives,0,17,1,3,dalit lowliv
1,1,gay people are a burden to our society,0,31,1,8,gay peopl burden societi
2,2,Arabs are not welcome,0,18,1,4,arab welcom
3,3,I'm not saying we should actually eliminate he...,0,79,2,17,say actual elimin heeb wish natur becam extinct
4,4,bananas are for black people,0,24,1,5,banana black peopl
...,...,...,...,...,...,...,...
41093,41139,Indian culture is so backwards,0,26,1,5,indian cultur backward
41094,41140,"Women don't want equality, they just want to b...",0,47,1,13,women want equal want charg
41095,41141,fuck covid,1,9,1,2,fuck covid
41096,41142,This computer is garbage,1,21,1,4,comput garbag


In [ ]:
data = df[['label','comment_transformed']]

In [ ]:
data

,label,comment_transformed
0,0,dalit lowliv
1,0,gay peopl burden societi
2,0,arab welcom
3,0,say actual elimin heeb wish natur becam extinct
4,0,banana black peopl
...,...,...
41093,0,indian cultur backward
41094,0,women want equal want charg
41095,1,fuck covid
41096,1,comput garbag


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize TfidfVectorizer with reduced max_features
max_features = 10000  # Adjust this value based on your dataset
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)

# Convert text data into TF-IDF vectors (sparse matrix)
X = tfidf_vectorizer.fit_transform(df['comment_transformed'])
y = df['label']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest classifier with default hyperparameters
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


Test Accuracy: 0.61338199513382


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import numpy as np 
# Initialize SGDClassifier with logistic loss function for binary classification
sgd_classifier = SGDClassifier(loss='log_loss')

# Define batch size
batch_size = 256  # You can adjust this value based on your memory constraints and dataset size

# Fit the classifier incrementally using partial_fit with specified batch size
for i in range(0, X_train.shape[0], batch_size):
    X_batch = X_train[i:i+batch_size]
    y_batch = y_train[i:i+batch_size]
    sgd_classifier.partial_fit(X_batch, y_batch, classes=np.unique(y_train))

# Make predictions on the test data
y_pred = sgd_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


Test Accuracy: 0.6731143552311436


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Initialize TfidfVectorizer with reduced max_features
max_features = 10000  # Adjust this value based on your dataset
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)

# Convert text data into TF-IDF vectors (sparse matrix)
X = tfidf_vectorizer.fit_transform(df['comment_transformed'])
y = df['label']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert sparse matrix to dense numpy array
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Initialize Gaussian Naive Bayes classifier
nb_classifier = GaussianNB()

# Fit the classifier to the training data
nb_classifier.fit(X_train_dense, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test_dense)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


Test Accuracy: 0.5396593673965937


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Initialize TfidfVectorizer with reduced max_features
max_features = 5000  # Adjust this value based on your dataset
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)

# Convert text data into TF-IDF vectors
X = tfidf_vectorizer.fit_transform(df['comment_transformed'])
y = df['label']

# Convert the sparse TF-IDF matrix to a dense numpy array
X = X.toarray()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential([
    Embedding(input_dim=max_features, output_dim=128),  # Embedding layer
    LSTM(32),  # LSTM layer with 32 units
    Dropout(0.5),  # Dropout layer to prevent overfitting
    Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 64  # Adjust batch size based on memory constraints
epochs = 5  # Reduce the number of epochs for faster training
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on test data
accuracy = model.evaluate(X_test, y_test)[1]
print("Test Accuracy:", accuracy)
